In [1]:
pip install ultralytics   

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pygame

In [6]:
from ultralytics import YOLO
import cv2
import pygame
import threading

# Initialize alarm system
pygame.mixer.init()
pygame.mixer.music.load("alarm.mp3")  # Your alarm sound file

# Function to play alarm sound in a separate thread
def play_alarm():
    if not pygame.mixer.music.get_busy():
        pygame.mixer.music.play()

# Harmful objects to detect
harmful_objects = {"knife", "gun", "weapon", "rifle", "pistol"}  # You can modify this list

# Load the YOLOv8 model (using yolov8s for speed)
model = YOLO("yolov8s.pt")

# Open the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 detection
    results = model.predict(source=frame, conf=0.5, verbose=False)

    # Extract detected classes
    detected_names = results[0].names
    detected_classes = results[0].boxes.cls.cpu().numpy()

    # Check for harmful object detection
    for cls_id in detected_classes:
        label = detected_names[int(cls_id)].lower()
        if label in harmful_objects:
            threading.Thread(target=play_alarm).start()  # Play alarm in a new thread
            break  # One alert per frame is enough

    # Show results
    annotated_frame = results[0].plot()
    cv2.imshow("YOLOv8 - Smart Surveillance", annotated_frame)

    # Exit on any key press
    if cv2.waitKey(1) != -1:
        break

cap.release()
cv2.destroyAllWindows()
